In [ ]:
import fitz  # PyMuPDF
from glob import glob

# Text chunking function to split text into manageable chunks
def chunk_text(text, chunk_size=1000):
    chunks = []
    while len(text) > chunk_size:
        # Split the text at the closest space to the chunk size to avoid cutting words
        split_point = text.rfind(' ', 0, chunk_size)
        chunks.append(text[:split_point])
        text = text[split_point:].strip()
    if text:
        chunks.append(text)
    return chunks

# Extract text from PDF files
text_lines = []

for file_path in glob("documents_RAG/*.pdf", recursive=True):
    doc = fitz.open(file_path)  # Open the PDF file
    pdf_text = ""
    for page in doc:
        pdf_text += page.get_text()  # Extract text from each page

    # Apply chunking to split the extracted text into smaller sections
    text_lines += chunk_text(pdf_text)


In [13]:
import ollama


def emb_text(text):
    response = ollama.embeddings(model="nomic-embed-text", prompt=text)
    return response["embedding"]


In [14]:
test_embedding = emb_text("This is a test")
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])


768
[1.0914263725280762, 0.5967336893081665, -3.9346163272857666, -0.6990123391151428, 1.5423402786254883, -0.13473758101463318, 0.8982678651809692, -0.46930229663848877, 0.9009982347488403, -0.6395869851112366]


In [15]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="http://localhost:19530")

collection_name = "my_rag_collection"


In [16]:
if milvus_client.has_collection(collection_name):
    milvus_client.drop_collection(collection_name)


In [17]:
milvus_client.create_collection(
    collection_name=collection_name,
    dimension=embedding_dim,
    metric_type="IP",  # Inner product distance
    consistency_level="Strong",  # Strong consistency level
)


In [18]:
from tqdm import tqdm

data = []

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": emb_text(line), "text": line})

milvus_client.insert(collection_name=collection_name, data=data)


Creating embeddings: 100%|██████████| 72/72 [00:04<00:00, 14.99it/s]


{'insert_count': 72, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71], 'cost': 0}